# RNA simulation from OpenMM force field

This notebook is red because:
- Sage has not been rigourously validated to work with Amber force fields
- Many of the functions used are experimental
- Small changes to this notebook can cause it to fail

In [ ]:
%env INTERCHANGE_EXPERIMENTAL=1

## Picking molecules

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3158920

In [ ]:
from utils import draw_molecule, nglview_show_openmm

In [ ]:
def molecule_from_rna_sequence(sequence):
    import random

    from rdkit import Chem
    from rdkit.Chem import AllChem

    from openff.toolkit import Molecule

    rna = Chem.MolFromSequence(sequence, flavor=2)

    for atom in rna.GetAtoms():
        if (
            atom.GetNumImplicitHs() == 1 
            and [a.GetSymbol() for a in atom.GetNeighbors()] == ["P"]
        ):
            atom.SetFormalCharge(-1)

        if (
            atom.GetChiralTag() == Chem.ChiralType.CHI_UNSPECIFIED
            and atom.GetSymbol() == "P"
        ):
            atom.SetChiralTag(random.choice([Chem.ChiralType.CHI_TETRAHEDRAL_CW, Chem.ChiralType.CHI_TETRAHEDRAL_CCW]))

    Chem.SanitizeMol(rna)
    offmol = Molecule.from_rdkit(rna)
    offmol.generate_conformers(n_conformers=1)

    return offmol


ribozyme = molecule_from_rna_sequence("GUGGC")
rna_substrate = molecule_from_rna_sequence("GCCU")

In [ ]:
from openff.toolkit import ForceField, Molecule, Topology

# PheAMP, phenylalanine adenosine monophosphate
pheamp = Molecule.from_smiles(
    "c1ccccc1C[C@H](N)C(=O)O[P@]([O-])(=O)OC[C@@H]2[C@@H](O)[C@@H](O)[C@@H](O2)N3C4N=CN=C(N)C=4N=C3",
    allow_undefined_stereo=False,
)
pheamp.generate_conformers(n_conformers=1)
pheamp.generate_unique_atom_names()
pheamp.visualize(show_all_hydrogens=False)

In [ ]:
from openff.units import unit

drawn_mol = ribozyme
draw_molecule(
    drawn_mol,
    width=1000,
    height=700,
    atom_notes={
        i: atom.formal_charge.m_as(unit.elementary_charge)
        for i, atom in enumerate(drawn_mol.atoms)
    },
)

## Parametrizing

In [ ]:
from openff.toolkit import ForceField

sage = ForceField("openff-2.1.1.offxml")
pheamp_interchange = sage.create_interchange(pheamp.to_topology())

In [ ]:
from tempfile import NamedTemporaryFile

import numpy as np
import openmm.app
from openff.interchange import Interchange
from openff.toolkit import Molecule, Topology
from rdkit import Chem


def parametrize_with_openmm(
    molecule,
    *,
    openmm_force_field,
    **kwargs,
):
    """
    Create an Interchange by parametrizing the molecule with an OpenMM force field
    """
    # OpenMM's force field templates can choke on RDKit hydrogens, 
    # so remove them and re-add them with OpenMM
    rdmol = Chem.RemoveHs(molecule.to_rdkit())

    # Convert the rdkit molecule to an openmm topology via PDB
    with NamedTemporaryFile("r") as f:
        Chem.rdmolfiles.MolToPDBFile(rdmol, f.name)
        omm_pdb = openmm.app.PDBFile(f.name)

    # Add the hydrogens back
    modeller = openmm.app.Modeller(omm_pdb.topology, omm_pdb.positions)
    modeller.addHydrogens(forcefield=openmm_force_field)

    # Put an appropriate box on the system so OpenMM doesn't get upset with PME
    positions_ary = np.asarray(modeller.positions.value_in_unit(openmm.unit.nanometer))
    box_length = np.abs(positions_ary.max(axis=0) - positions_ary.min(axis=0)).max() + 1
    modeller.topology.setPeriodicBoxVectors(
        [[box_length, 0, 0], [0, box_length, 0], [0, 0, box_length]]
        * openmm.unit.nanometer
    )

    # Create an OpenFF molecule so we can preserve the RDMol's chemistry in the final interchange
    off_mol = Molecule.from_rdkit(rdmol, allow_undefined_stereo=True)

    # Prepare an OpenFF Topology from the OFF Molecule and the new hydrogens
    with NamedTemporaryFile("r") as f:
        openmm.app.PDBFile.writeFile(modeller.topology, modeller.positions, f.name)
        off_top = Topology.from_pdb(f.name, unique_molecules=[off_mol])

    # Create the OpenMM System
    system = openmm_force_field.createSystem(
        modeller.topology,
        **kwargs,
    )

    # Create the Interchange
    interchange = Interchange.from_openmm(
        topology=off_top,
        system=system,
        positions=modeller.positions,
    )

    # Don't expose the box we chose
    interchange.box = None
    interchange.topology.box_vectors = None

    return interchange


kwargs = {
    "openmm_force_field": openmm.app.ForceField(
        # "amber14/protein.ff14SB.xml",
        "amber14/RNA.OL3.xml",
        # "amber14/DNA.OL15.xml",
        # "amber14/lipid17.xml",
        # "amber14/GLYCAM_06j-1.xml",
        "amber14/tip3p.xml",
    ),
    "nonbondedMethod": openmm.app.PME,
    "nonbondedCutoff": 0.9 * openmm.unit.nanometer,
    "constraints": openmm.app.HBonds,
    "rigidWater": True,
}

ribozyme_interchange = parametrize_with_openmm(
    ribozyme,
    **kwargs,
)
rna_substrate_interchange = parametrize_with_openmm(
    rna_substrate,
    **kwargs,
)

rna_substrate_interchange.visualize()

## Combining and solvating

In [ ]:
from openff.units import unit

# Move all molecule's minimum corner to the origin
pheamp_interchange.positions -= pheamp_interchange.positions.min(axis=0)
rna_substrate_interchange.positions -= rna_substrate_interchange.positions.min(axis=0)
ribozyme_interchange.positions -= ribozyme_interchange.positions.min(axis=0)

# Find the limits of each molecule
pheamp_bb = pheamp_interchange.positions.max(axis=0)
rna_substrate_bb = rna_substrate_interchange.positions.max(axis=0)
ribozyme_bb = ribozyme_interchange.positions.max(axis=0)

# Move the two RNA molecules
stack_along_dim = pheamp_bb.argmin()
rna_substrate_interchange.positions[:, stack_along_dim] += (
    pheamp_bb[stack_along_dim] + 1.5 * unit.angstrom
)
ribozyme_interchange.positions[:, stack_along_dim] -= (
    ribozyme_bb[stack_along_dim] + 1.5 * unit.angstrom
)

# Choose a box that fits everything
DODECAHEDRON = np.array(
    [
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.5, 0.5, np.sqrt(2.0) / 2.0],
    ]
)
max_dist = (
    pheamp_bb[stack_along_dim]
    + rna_substrate_bb[stack_along_dim]
    + ribozyme_bb[stack_along_dim]
)
buffer = 2 * unit.nanometer
rna_substrate_interchange.box = (max_dist + buffer) * DODECAHEDRON
pheamp_interchange.box = rna_substrate_interchange.box
ribozyme_interchange.box = rna_substrate_interchange.box

In [ ]:
combined_interchange = (
    pheamp_interchange + rna_substrate_interchange + ribozyme_interchange
)

combined_interchange.visualize()

In [ ]:
from tempfile import NamedTemporaryFile

from pdbfixer import PDBFixer

with NamedTemporaryFile("r") as f:
    combined_interchange.to_pdb(f.name)
    fixer = PDBFixer(f.name)

fixer.topology = combined_interchange.topology.to_openmm()
fixer.positions = combined_interchange.positions.to_openmm()

fixer.addSolvent(
    boxVectors=combined_interchange.box.to_openmm(),
    ionicStrength=0.1 * openmm.unit.molar,
)

with NamedTemporaryFile("r") as f:
    openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f.name)
    solvated_topology = Topology.from_pdb(
        f.name, unique_molecules=combined_interchange.topology.unique_molecules
    )

w = nglview_show_openmm(fixer.topology, fixer.positions)
w.add_line()
w

In [ ]:
total_charge = np.sum([mol.total_charge.m for mol in solvated_topology.molecules])
neutralised_molecules = []
for molecule in solvated_topology.molecules:
    if total_charge < 0 and molecule.is_isomorphic_with(Molecule.from_smiles("[Cl-]")):
        total_charge += 1
    elif total_charge > 0 and molecule.is_isomorphic_with(Molecule.from_smiles("[Na+]")):
        total_charge -= 1
    else:
        neutralised_molecules.append(molecule)

neutralised_topology = Topology.from_molecules(neutralised_molecules)

In [ ]:
# Parametrize the solvent, then add to the combined interchange
solvent_interchange = sage.create_interchange(
    Topology.from_molecules([*neutralised_topology.molecules][3:]),
)
solvent_interchange.box = fixer.topology.getPeriodicBoxVectors()
solvated_interchange = combined_interchange + solvent_interchange

solvated_interchange.visualize()

## Simulate!

In [ ]:
solvated_interchange.minimize()

In [ ]:
temperature = 300 * unit.kelvin
friction_coefficient = 1 / unit.picosecond
time_step = 2 * unit.femtosecond

simulation = solvated_interchange.to_openmm_simulation(
    integrator=openmm.LangevinMiddleIntegrator(
        temperature.to_openmm(),
        friction_coefficient.to_openmm(),
        time_step.to_openmm(),
    )
)

# Add a reporter to record the structure every few steps
dcd_reporter = openmm.app.DCDReporter(file="rna.dcd", reportInterval=1000)
simulation.reporters.append(dcd_reporter)

simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1 * openmm.unit.minute)

In [ ]:
w = nglview_show_openmm(simulation.topology, "rna.dcd")
w.add_licorice(selection="not (HOH NA+ CL-)")
w.add_spacefill(selection="HOH NA+ CL-", opacity=0.05)
w.add_unitcell()
w